# Topic modeling and visualization of tweets

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import pyLDAvis.gensim_models
import seaborn as sns
import os
import sys

/home/andenet/dev/projects/ml/Twitter-Data-Analysis/twitter_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
sys.path.append(".")
sys.path.append("..")

In [3]:
#local import
from text_cleaner import TextCleaner
from build_model import BuildModel
from visualize import Visualize

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## Load The Data 

In [4]:
tweets_df = pd.read_csv("../data/processed_tweet_data.csv")

In [5]:
tweets_df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Sun Aug 07 22:31:20 +0000 2022,Twitter for Android,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,en,15760,2,i_ameztoy,20497,2621,NaN,"[{'text': 'City', 'indices': [132, 137]}]","[{'screen_name': 'i_ameztoy', 'name': 'Iban Am...",NaN
1,Sun Aug 07 22:31:16 +0000 2022,Twitter for Android,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,en,6967,201,ZIisq,65,272,NaN,"[{'text': 'China', 'indices': [18, 24]}, {'tex...","[{'screen_name': 'IndoPac_Info', 'name': 'Indo...",NaN
2,Sun Aug 07 22:31:07 +0000 2022,Twitter for Android,"China even cut off communication, they don't a...",0.000000e+00,0.000000,en,2166,0,Fin21Free,85,392,NaN,"[{'text': 'XiJinping', 'indices': [127, 137]}]","[{'screen_name': 'ZelenskyyUa', 'name': 'Волод...",Netherlands
3,Sun Aug 07 22:31:06 +0000 2022,Twitter for Android,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,en,2166,0,Fin21Free,85,392,NaN,"[{'text': 'XiJinping', 'indices': [9, 19]}]",[],Netherlands
4,Sun Aug 07 22:31:04 +0000 2022,Twitter for iPhone,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,en,17247,381,VizziniDolores,910,2608,NaN,[],"[{'screen_name': 'ChinaUncensored', 'name': 'C...","Ayent, Schweiz"


In [6]:
# Let's look at a sample tweet
tweets_df['original_text'][100]

'RT @anku5hdilraaj_: I guess #WWIII on its way for #Taiwan https://t.co/oomVltBmKF'

# Standardize tweets
To utilize the power of machine learning algorithms, we need to provide the data translated into a meaningful features. 
Here we're going to do:
1. Word capitalization (string class functions)
2. Punctuation (regular expressions)
3. Singular-plural versions of same word (lemmatization)
4. Common words like 'and' (stopwords)

We will use the `NLTK` and `re` packages to clean the text and `gensim` to implement various learning algorithms.

In [6]:
text_cleaner = TextCleaner(tweets_df['original_text'].tolist())
text_cleaner.filterTweetList()

In [7]:
tweets_df['original_text'][100]

'RT @anku5hdilraaj_: I guess #WWIII on its way for #Taiwan https://t.co/oomVltBmKF'

Now let's see the standardized version of the above tweet

In [8]:
text_cleaner.tweetList[100]

['guess', '#wwiii', 'way', '#taiwan']

In [9]:
clean_tweets = text_cleaner.tweetList

### Now the data are ready for processing.
Many algorithms use a similar initial format, which is to

1. Build a dictionary with all words in the dataset
2. Store the word counts (using above dictionary) of each tweet in a corpus
Note that in step 2 only the word frequency is used. This is a so-called "bag-of-words" approach, which does not account for ordering of words next to each other. Other analyses like bigrams or trigrams could be used if word ordering was highly conserved.

Because I am looking at unlabeled data, and I want to get intuition for the data, I chose to use Latent Dirichlet Allocation (LDA), a topic modeling approach that probablistically learns the latent (unobserved) topics of a group of documents. There are a couple other algorithms, like LSA, LSI, or TF-IDF, which were either less accurate at predicting similarity or more suited to supervised learning.

In [10]:
buildModel = BuildModel()

Create a model objects

In [11]:
twtDict = buildModel.makeDict(clean_tweets)
twtCorpus = buildModel.makeCorpus(clean_tweets, twtDict)
twtLda = buildModel.createLDA(twtCorpus, twtDict)

Save model objects

In [12]:
#Save model object
buildModel.saveModelObjects(twtLda, 'twtLDAmodel', twtCorpus, 'twtCorpus.mm', twtDict, 'twtDictionary.dict' )

Model Objects Successfully SAVED!


## Topic Visualization of tweets on US-China Conflict using LDA

we can visualize our results using the `pyLDAvis` package. The plot is interactive. Each circle is a topic and the size represents the abundance of that topic in the corpus. Along with each topic are the associated words that go with it.

In [18]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(twtLda, twtCorpus, twtDict)

/home/andenet/dev/projects/ml/Twitter-Data-Analysis/twitter_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [19]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10    -0.255246  0.006367       1        1  4.742327
20     0.003357 -0.013988       2        1  4.004059
24     0.024674 -0.023435       3        1  3.491239
23    -0.074461  0.183014       4        1  3.154751
48    -0.232133  0.075760       5        1  3.053769
42     0.030163  0.117812       6        1  2.558388
47     0.050375  0.262021       7        1  2.554038
44     0.084286 -0.025931       8        1  2.516174
41    -0.048029 -0.070461       9        1  2.421619
28     0.111119 -0.060198      10        1  2.382079
45    -0.145008 -0.002778      11        1  2.374672
11     0.122879 -0.006334      12        1  2.299777
13    -0.184626 -0.014541      13        1  2.267632
35    -0.139057 -0.048680      14        1  2.189089
38     0.086464 -0.069016      15        1  2.147707
29     0.084674  0.130195      16        1  2.141599
46     0.040723 -0.013035      17        1  2.079106
5     -0.020628 -0.114427      18        1  2.078519
0      0.074753  0.137615      19        1  2.037050
15     0.051924 -0.109708      20        1  2.012022
27     0.084511  0.010013      21        1  1.966197
43    -0.085057 -0.056074      22        1  1.894102
2     -0.067618  0.048184      23        1  1.878004
34     0.049642  0.031199      24        1  1.852071
36    -0.109550  0.010048      25        1  1.837485
14     0.009799 -0.087196      26        1  1.830127
21     0.026513 -0.056607      27        1  1.797226
18    -0.002512 -0.112749      28        1  1.781893
4      0.067096 -0.068665      29        1  1.779642
7     -0.003327  0.035680      30        1  1.747015
9     -0.083703  0.013945      31        1  1.679302
33     0.008059 -0.048593      32        1  1.650295
31     0.110257  0.114844      33        1  1.649536
26    -0.065497  0.117463      34        1  1.646636
39     0.065843 -0.020498      35        1  1.645905
16    -0.034056 -0.082041      36        1  1.558022
6      0.031587 -0.047225      37        1  1.554838
32    -0.023488  0.030020      38        1  1.523515
30     0.049347 -0.052951      39        1  1.519777
8      0.016179 -0.023594      40        1  1.488255
12     0.050131 -0.080148      41        1  1.484598
22    -0.057651 -0.090725      42        1  1.467768
3      0.063152 -0.001867      43        1  1.429205
19    -0.064500 -0.040646      44        1  1.397885
25     0.073315  0.029001      45        1  1.366035
17     0.063764  0.057023      46        1  1.360094
49     0.071605 -0.036297      47        1  1.283416
1      0.051526 -0.040049      48        1  1.273695
37     0.009648  0.102237      49        1  1.199005
40     0.028782  0.006019      50        1  0.952841, topic_info=          Term         Freq        Total Category  logprob  loglift
53      taiwan  3469.000000  3469.000000  Default  30.0000  30.0000
59     country  1803.000000  1803.000000  Default  29.0000  29.0000
19       drill  1714.000000  1714.000000  Default  28.0000  28.0000
219     island  1496.000000  1496.000000  Default  27.0000  27.0000
82        part  1271.000000  1271.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
234  democracy    19.020001   272.669821  Topic50  -4.8877   1.9907
213        amp    25.122715  1974.833601  Topic50  -4.6094   0.2890
219     island    23.925286  1496.811161  Topic50  -4.6582   0.5173
104      visit    24.750979  2013.110827  Topic50  -4.6243   0.2549
77      #china    22.530563  4420.853992  Topic50  -4.7183  -0.6257

[2892 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
4124      5  0.891999  "#cambodia
495       5  0.969151       "#pla
224       2  0.992147    "#taiwan
224      28  0.001629    "#taiwan
993      37  0.927388    "#trkiye
...     ...       ...         ...
933      34  0.008337           |
933      38  0.008337           |
933    

## Visualization-guided analysis
After finding a topic of interest, we can find sort the data based on this topic for further information on which country got the most tweets in a given topic. To do this, we first need to match the indices between teh visualization and our LDA model.